In [32]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
import pandas as pd


In [ ]:
file_path = None
df = pd.read_csv(file_path, delimiter='\s+', header=None)
columns = ['Label'] + [f'angle_idx_{i-1}' for i in range(1, df.shape[1])]
df.columns = columns

numpy_array = df.to_numpy()
X_train = df.to_numpy()[:,2:]
Y_train = None

X_test = None
Y_test = None

# need to fix data generation

In [37]:
class CustomDataset(Dataset):
    def __init__(self, lidar, actions):
        self.lidar = lidar
        self.actions = actions
    def __len__(self):
        return len(self.lidar)
    def __getitem__(self, idx):
        lidar_tensor = torch.tensor(self.lidar[idx], dtype=torch.float32)
        action = self.actions[idx]
        return lidar_tensor, action

In [38]:
train_dataset = CustomDataset(X_train, Y_train)
test_dataset = CustomDataset(X_test, Y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [29]:
class ControlNN(nn.Module):
    def __init__(self, input_size=24, output_size=4, hidden_layers=10, hidden_size=64):
        super(ControlNN, self).__init__()

        layers =  []

        layers.append(nn.Linear(input_size, hidden_size))
        layers.append(nn.ReLU())

        for i in range(hidden_layers-1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)    


In [33]:
def train_model(model, train_loader, test_loader, device, learning_rate=0.001, num_epochs=30):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_samples = 0
        for lidar, action in train_loader:
            lidar, actions = lidar.to(device), action.to(device)
            optimizer.zero_grad()
            outputs = model(lidar)
            loss = criterion(outputs, actions)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * lidar.size(0)
            num_samples += lidar.size(0)

        avg_loss = running_loss / num_samples
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")
    
    model.eval() 
    
    total_loss = 0.0
    total_abs_error = 0.0
    num_samples = 0

    criterion = nn.MSELoss()

    with torch.no_grad():
        for lidar, actions in test_loader:
            lidar, actions = lidar.to(device), actions.to(device)
            
            outputs = model(lidar)

            mse_loss = criterion(outputs, actions)
            total_loss += mse_loss.item() * lidar.size(0)
            total_abs_error += torch.abs(outputs - actions).sum().item() 

            num_samples += lidar.size(0)

    avg_mse_loss = total_loss / num_samples
    avg_abs_error = total_abs_error / num_samples
    
    print(f"Test MSE Loss: {avg_mse_loss:.4f}")
    print(f"Test Mean Absolute Error (MAE): {avg_abs_error:.4f}")
    return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ControlNN(device)
model = train_model(model, train_loader, test_loader, device)